In [1]:
import numpy as np
import h5py

import pandas as pd
from utils import preprocess
from collections import defaultdict
import string

import sklearn
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, Lambda
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from gensim.models import word2vec

from utils import preprocess
np.random.seed(1234)

Using TensorFlow backend.


In [9]:
def create_docs(df, n_gram_max=1):
    docs = []
    special_latters = set('à ñ Ν ê Υ Å Æ ö δ è Π α é ä ë æ ô ç ü ἶ Ο â î ï Σ'.split()) - set(string.ascii_uppercase) - set(string.ascii_lowercase) - set(',.:!;"\'?')
    for i, text in enumerate(df.text):    
        def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams

        doc = preprocess(text).split()
        
        prod = special_latters & set(text)
        special_chars = ''
        if prod:
            for c in prod:
                freq = text.count(c)
                special_chars += (' {} '.format(c) * freq)            
                
        docs.append(' '.join(add_ngram(doc, n_gram_max)) + special_chars)
        
    return docs

In [14]:
df = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
text = df.text.values
text_test = df_test.text.values

author2class = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
class2author = ['EAP', 'HPL', 'MWS']
y = np.array([author2class[a] for a in df.author])
y = to_categorical(y)

In [15]:
embedding_dims = 20

In [16]:
def create_model(input_dim, embeddings_dims=20):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [17]:
raw_docs = create_docs(df, n_gram_max=1)
raw_docs_test = create_docs(df_test, n_gram_max=1)

seed = 7
num_split = 5
epochs = 45

# for next training
predict_prob_features = np.zeros((len(df), 3))
predict_prob_features_test = np.zeros((len(df_test), 3))

ite = 0
sum_loss = 0.
min_count = 1

kf = KFold(n_splits=num_split, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(text):
    ite += 1
    print('{}/{}: #Trains: {}, #Val: {}'.format(ite, num_split, len(train_index), len(val_index)), end=' ')
    
    docs_train = [raw_docs[i] for i in train_index]
    docs_val = [raw_docs[i] for i in val_index]

    # get vocab
    tokenizer = Tokenizer(filters='', lower=False)
    #     tokenizer.fit_on_texts(docs_train)
    tokenizer.fit_on_texts(docs_train + docs_val)

    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

    tokenizer = Tokenizer(num_words=num_words, filters='', lower=False)
    #     tokenizer.fit_on_texts(docs_train)
    tokenizer.fit_on_texts(docs_train + docs_val)    
    
    docs_train = tokenizer.texts_to_sequences(docs_train)
    docs_val = tokenizer.texts_to_sequences(docs_val)
    docs_test = tokenizer.texts_to_sequences(raw_docs_test)

    maxlen = 256# max([len(i) for i in  docs_train] + [len(i) for i in  docs_val])
    x_train = pad_sequences(sequences=docs_train, maxlen=maxlen)
    x_val = pad_sequences(sequences=docs_val, maxlen=maxlen)
    x_test = pad_sequences(sequences=docs_test, maxlen=maxlen)

    y_train, y_val = y[train_index], y[val_index]

    input_dim = max(np.max(x_train), np.max(x_val)) + 1
#     input_dim = np.max(np.max(x_train) + 1    
    print('#vocab: {} '.format(num_words), end=' ')
    print(x_train.shape, x_val.shape, x_test.shape)
    

    model = create_model(input_dim)
    
    checkpointer = ModelCheckpoint(filepath='./fasttext_weights/weights.hdf5', verbose=0, save_best_only=True)

    hist = model.fit(x_train, y_train,
                     batch_size=16,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     callbacks=[EarlyStopping(patience=4, monitor='val_loss'), 
                                checkpointer])

    # load best weights
    model.load_weights('./fasttext_weights/weights.hdf5')
    y_pred = model.predict_proba(x_val)
    sum_loss += log_loss(y_pred=y_pred, y_true=np.nonzero(y_val)[1])

    # save features
    predict_prob_features[val_index] = y_pred
    predict_prob_features_test += model.predict_proba(x_test)

    print('valLoss: {}'.format(sum_loss/ite))


1/5: #Trains: 15663, #Val: 3916 #vocab: 28285  (15663, 256) (3916, 256) (8392, 256)
Train on 15663 samples, validate on 3916 samples
Epoch 1/45
 3904/15663 [======>.......................] - ETA: 15s - loss: 1.0865 - acc: 0.4068

KeyboardInterrupt: 

In [ ]:
for a, c in author2class.items():
    df['{}_fasttext_ngram1'.format(a)] = predict_prob_features[:, c]
    df_test['{}_fasttext_ngram1'.format(a)] = predict_prob_features_test[:, c]/num_split

In [ ]:
# df.to_csv('./data/train_feature.csv_')
# df_test.to_csv('./data/test_feature.csv_')